In [1]:
import pandas as pd
import numpy as np

In [8]:
path='D:\\project\\TianChi\\Data\\'
train_data = pd.read_csv(path+'train.csv')
test_data = pd.read_csv(path+'testA.csv')
print("Trian data shape: ", train_data.shape)
print("Test data shape: ", test_data.shape)
train_data.head()

Trian data shape:  (100000, 3)
Test data shape:  (20000, 2)


,id,heartbeat_signals,label
0,0,"0.9912297987616655,0.9435330436439665,0.764677...",0.0
1,1,"0.9714822034884503,0.9289687459588268,0.572932...",0.0
2,2,"1.0,0.9591487564065292,0.7013782792997189,0.23...",2.0
3,3,"0.9757952826275774,0.9340884687738161,0.659636...",0.0
4,4,"0.0,0.055816398940721094,0.26129357194994196,0...",2.0


In [27]:
# 整理数据类型，降低内存使用
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum()/ 1024**2
    print(f"Memory usage of dataframe is {round(start_mem,2)}MB.")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after optimization is {round(end_mem,2)}MB.")
    print('Decreased by {:.2f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

### 数据预处理

In [28]:
def format_data(df, dtype):
    data_lst = []

    if dtype == 'train':
        for items in df.values:
            data_lst.append([items[0]] + [float(i) for i in items[1].split(',')] + [items[2]])
        df = pd.DataFrame(np.array(data_lst),columns = ['id']+['s_'+str(i) for i in range(len(data_lst[0])-2)]+['label'] )
    else:
        for items in df.values:
            data_lst.append([items[0]] + [float(i) for i in items[1].split(',')])
        df = pd.DataFrame(np.array(data_lst),columns = ['id']+['s_'+str(i) for i in range(len(data_lst[0])-1)])
    df = reduce_mem_usage(df)
    return df

train = format_data(train_data, 'train')
test = format_data(test_data, 'test')
test.head()

Memory usage of dataframe is 157.93MB.
Memory usage of dataframe after optimization is 39.67MB.
Decreased by 74.88%
Memory usage of dataframe is 31.43MB.
Memory usage of dataframe after optimization is 7.9MB.
Decreased by 74.88%


,id,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,...,s_195,s_196,s_197,s_198,s_199,s_200,s_201,s_202,s_203,s_204
0,100000.0,0.991699,1.000000,0.631836,0.136230,0.041412,0.102722,0.120850,0.123413,0.107910,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,100001.0,0.607422,0.541504,0.340576,0.000000,0.090698,0.164917,0.195068,0.168823,0.198853,...,0.389893,0.386963,0.367188,0.364014,0.360596,0.357178,0.350586,0.350586,0.350586,0.36377
2,100002.0,0.975098,0.670898,0.686523,0.708496,0.718750,0.716797,0.720703,0.701660,0.596680,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,100003.0,0.995605,0.916992,0.520996,0.000000,0.221802,0.404053,0.490479,0.527344,0.518066,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,100004.0,1.000000,0.888184,0.745605,0.531738,0.380371,0.224609,0.091125,0.057648,0.003914,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [29]:
# 训练/测试数据准备
x_train = train.drop(['id','label'], axis=1)
y_train = train['label']
x_test = test.drop(['id'], axis=1)

### 模型训练

In [30]:
def abs_sum(y_pre, y_tru):
    y_pre = np.array(y_pre)
    y_tru = np.array(y_tru)
    loss = sum(sum(abs(y_pre - y_tru)))
    return loss


In [ ]:
import xgboost as xgb

from sklearn.model_selection import train_test_split, KFold

In [ ]:

def cv_model(clf, train_x, train_y, test_x, clf_name):
    folds = 5
    seed = 2021
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    test = np.zeros((test_x.shape[0], 4))
    
    cv_scores = []
    